<h1>Data Preprocessing</h1>

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Downloading NLTK resources (only need to run this once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Loading the training and validation data
training_data = pd.read_csv('twitter_training.csv')
validation_data = pd.read_csv('twitter_validation.csv')

training_data.head()

[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading stopwords: <urlopen error [Errno 61]
[nltk_data]     Connection refused>
[nltk_data] Error loading wordnet: <urlopen error [Errno 61]
[nltk_data]     Connection refused>


,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [3]:
#Renaming the columns
training_data.columns = ['id', 'topic', 'sentiment', 'tweet']
validation_data.columns = ['id', 'topic', 'sentiment', 'tweet']
training_data.head()

,id,topic,sentiment,tweet
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [4]:
# Function to preprocess the text data
def preprocess_tweet(tweet):
    # Handling NaN values
    if isinstance(tweet, float) and np.isnan(tweet):
        return ""
    
    # Converting text to lowercase
    tweet = tweet.lower()
    
    # Removing special characters, URLs, and mentions
    tweet = re.sub(r'http\S+', '', tweet) # that\S is a regular expression that means "any non-whitespace character"
    tweet = re.sub(r'@[A-Za-z0-9]+', '', tweet) #+ means "one or more"
    tweet = re.sub(r'[^a-zA-Z\s]', '', tweet) #^ means "not"
    
    return tweet

# Preprocessing the text data
training_data['tweet'] = training_data['tweet'].apply(preprocess_tweet)
validation_data['tweet'] = validation_data['tweet'].apply(preprocess_tweet)

<h1>Model Development</h1>

In [5]:
# Initializing TF-IDF vectorizer with adjusted parameters
tfidf_vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')  

# Fitting and transforming the text data
X_train = tfidf_vectorizer.fit_transform(training_data['tweet'])
X_test = tfidf_vectorizer.transform(validation_data['tweet'])

# Encoding using Label Encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
training_data['sentiment'] = le.fit_transform(training_data['sentiment'])
validation_data['sentiment'] = le.fit_transform(validation_data['sentiment'])

y_train = training_data['sentiment']
y_test = validation_data['sentiment']

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42) 
model.fit(X_train, y_train)

# Predicting on validation data
y_pred = model.predict(X_test)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.963963963963964
